In [8]:
import pandas as pd
import numpy as np
# import math
# import matplotlib.pyplot as plt # Visualization 
# import matplotlib.dates as mdates # Formatting dates
# import seaborn as sns # Visualization
# from sklearn.preprocessing import MinMaxScaler
# import torch # Library for implementing Deep Neural Network 
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader

c:\Users\user\anaconda3\envs\cuda116\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 미세먼지농도 = 미세먼지, 초미세먼지농도 = 초미세먼지

In [79]:
import os
for x in range(len(os.listdir('data'))):
    if x == 0: df = pd.read_csv(f"./data/{os.listdir('data/')[x]}", encoding="cp949").rename(columns={'미세먼지농도(㎍/㎥)':'미세먼지(㎍/㎥)','초미세먼지농도(㎍/㎥)':'초미세먼지(㎍/㎥)'})
    else: df = pd.concat([df, pd.read_csv(f"./data/{os.listdir('data/')[x]}", encoding="cp949").rename(columns={'미세먼지농도(㎍/㎥)':'미세먼지(㎍/㎥)','초미세먼지농도(㎍/㎥)':'초미세먼지(㎍/㎥)'})], axis=0)
df = df.reset_index().drop(columns=['index'])

In [81]:
df

,측정일시,권역명,측정소명,이산화질소농도(ppm),오존농도(ppm),일산화탄소농도(ppm),아황산가스농도(ppm),미세먼지(㎍/㎥),초미세먼지(㎍/㎥)
0,20190101,동남권,강남구,0.0270,0.0130,0.60,0.0050,35.0,30.0
1,20190101,동남권,강동구,0.0370,0.0120,0.70,0.0050,40.0,30.0
2,20190101,동북권,강북구,0.0240,0.0200,0.60,0.0030,44.0,28.0
3,20190101,서남권,강서구,0.0290,0.0170,0.50,0.0050,46.0,24.0
4,20190101,서남권,관악구,0.0350,0.0160,0.50,0.0050,39.0,28.0
...,...,...,...,...,...,...,...,...,...
45645,20231231,도심권,용산구,0.0286,0.0146,0.78,0.0025,22.0,15.0
45646,20231231,서북권,은평구,0.0172,0.0178,1.12,0.0025,21.0,18.0
45647,20231231,도심권,종로구,0.0238,0.0180,0.73,0.0029,22.0,18.0
45648,20231231,도심권,중구,0.0261,0.0179,0.67,0.0028,20.0,18.0


In [82]:
print(df.dtypes, end ='\n\n')
print(df.isnull().sum(), end ='\n\n')
print(len(df))
df.describe()

측정일시              int64
권역명              object
측정소명             object
이산화질소농도(ppm)    float64
오존농도(ppm)       float64
일산화탄소농도(ppm)    float64
아황산가스농도(ppm)    float64
미세먼지(㎍/㎥)       float64
초미세먼지(㎍/㎥)      float64
dtype: object

측정일시              0
권역명               0
측정소명              0
이산화질소농도(ppm)    450
오존농도(ppm)       340
일산화탄소농도(ppm)    755
아황산가스농도(ppm)    498
미세먼지(㎍/㎥)       394
초미세먼지(㎍/㎥)      350
dtype: int64

45650


,측정일시,이산화질소농도(ppm),오존농도(ppm),일산화탄소농도(ppm),아황산가스농도(ppm),미세먼지(㎍/㎥),초미세먼지(㎍/㎥)
count,4.565000e+04,45200.000000,45310.000000,44895.000000,45152.000000,45256.000000,45300.000000
mean,2.021066e+07,0.023260,0.027450,0.470845,0.003160,37.057274,20.649912
std,1.414473e+04,0.012145,0.013503,0.182918,0.000986,26.936450,14.554805
min,2.019010e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.020040e+07,0.014000,0.017700,0.340000,0.002600,22.000000,11.000000
50%,2.021070e+07,0.020000,0.026000,0.400000,0.003000,32.000000,17.000000
75%,2.022100e+07,0.030100,0.036000,0.590000,0.004000,45.000000,26.000000
max,2.023123e+07,0.089000,0.097000,2.000000,0.010000,503.000000,153.000000


In [102]:
# df = df.interpolate(method='nearest', order = 5)
# df = df.interpolate(method='linear', order = 5)
df = df.interpolate(method='quadratic', order = 5)

ValueError: Invalid fill method. Expecting pad (ffill) or backfill (bfill). Got quadratic

In [103]:
df['측정일시'] = pd.to_datetime(df['측정일시'], format="%Y%m%d")
# df['측정년도'] = df['측정일시'].dt.year
# df['측정월'] = df['측정일시'].dt.month
# df['측정일'] = df['측정일시'].dt.day

In [121]:
merge_df = df.groupby('측정일시')['이산화질소농도(ppm)','오존농도(ppm)','일산화탄소농도(ppm)','아황산가스농도(ppm)','미세먼지(㎍/㎥)','초미세먼지(㎍/㎥)'].mean().reset_index()

C:\Users\user\AppData\Local\Temp\ipykernel_30364\912483302.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  merge_df = df.groupby('측정일시')['이산화질소농도(ppm)','오존농도(ppm)','일산화탄소농도(ppm)','아황산가스농도(ppm)','미세먼지(㎍/㎥)','초미세먼지(㎍/㎥)'].mean().reset_index()


In [123]:
merge_df.to_csv('./data/서울시_평균_대기정보_19-23.csv',encoding='cp949')

In [124]:
merge_df

,측정일시,이산화질소농도(ppm),오존농도(ppm),일산화탄소농도(ppm),아황산가스농도(ppm),미세먼지(㎍/㎥),초미세먼지(㎍/㎥)
0,2019-01-01,0.030120,0.015680,0.6720,0.004320,37.64,24.76
1,2019-01-02,0.036480,0.011800,0.7040,0.004680,34.40,22.20
2,2019-01-03,0.045360,0.006680,0.7720,0.005120,39.20,23.88
3,2019-01-04,0.058920,0.003640,1.0560,0.006080,59.88,41.56
4,2019-01-05,0.030080,0.016440,0.6600,0.005160,64.20,40.92
...,...,...,...,...,...,...,...
1821,2023-12-27,0.053304,0.006612,1.0244,0.003624,74.60,56.32
1822,2023-12-28,0.044608,0.008467,0.8852,0.003556,52.72,41.20
1823,2023-12-29,0.038368,0.010398,0.7432,0.003152,42.12,32.24
1824,2023-12-30,0.034380,0.008840,0.6908,0.002532,38.56,31.20
